In [4]:
import sqlite3

conn = sqlite3.connect("./database/database.db")
cur = conn.cursor()
sql = "DELETE FROM T車両台帳;"
cur.execute(sql)
conn.commit()
sql = """
    SELECT
    company_use_number,
    classification,
    maker,
    model_number,
    body_number,
    modelYear,
    car_size,
    load_capacity,
    specification,
    riding_capacity,
    fuel_type,
    update_date
    FROM TBase_Vehicle
    ORDER BY id;
    """
cur.execute(sql)
result = cur.fetchall()
i = 0
for c in result:
    i += 1
    sql = f"INSERT INTO T車両台帳 VALUES({i}, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);"
    cur.execute(sql, c)
    conn.commit()

In [ ]:
import sqlite3

conn = sqlite3.connect("database/database.db")
cur = conn.cursor()
sql = """
DELETE FROM T車両台帳 WHERE company_use_number = '123';
DELETE FROM T登録番号 WHERE company_use_number = '123';
DELETE FROM T車両履歴 WHERE company_use_number = '123';
"""
cur.executescript(sql)
conn.commit()

In [3]:
import sqlite3

# テーブルバックアップ
conn = sqlite3.connect("database/database.db")
cur = conn.cursor()
sql = """
    DELETE FROM TBK車両台帳;
    DELETE FROM TBK車両履歴;
    DELETE FROM TBK登録番号;
    INSERT INTO TBK車両台帳 SELECT * FROM T車両台帳;
    INSERT INTO TBK車両履歴 SELECT * FROM T車両履歴;
    INSERT INTO TBK登録番号 SELECT * FROM T登録番号;
    """
cur.executescript(sql)
conn.commit()

In [1]:
import sqlite3

# バックアップデータ戻し
conn = sqlite3.connect("database/database.db")
cur = conn.cursor()
sql = """
    DELETE FROM T車両台帳;
    DELETE FROM T車両履歴;
    DELETE FROM T登録番号;
    INSERT INTO T車両台帳 SELECT * FROM TBK車両台帳;
    INSERT INTO T車両履歴 SELECT * FROM TBK車両履歴;
    INSERT INTO T登録番号 SELECT * FROM TBK登録番号;
    """
cur.executescript(sql)
conn.commit()

In [ ]:
import sqlite3

# TW系テーブルクリア
conn = sqlite3.connect(r"database\database.db")
cur = conn.cursor()
sql = """
    DELETE FROM TPrintWork;
    DELETE FROM TW内訳別申請台数;
    DELETE FROM TW内訳別保有台数;
    DELETE FROM TW車格別保有台数;
    DELETE FROM TW申請車両;
    """
cur.executescript(sql)
conn.commit()

In [9]:
import datetime
from dateutil.relativedelta import relativedelta

str_date = '2023/05/01'
target_date = datetime.datetime.strptime(str_date, '%Y/%m/%d')
result_date = target_date + relativedelta(months=-3)
three_month_ago = str(format(result_date, '%Y/%m/%d'))
print(three_month_ago)

2023/02/01


In [16]:
import sqlite3

conn = sqlite3.connect(r"database\database.db")
cur = conn.cursor()
sql = """
    SELECT circumstances FROM T車両履歴
    WHERE department = '03'
    AND implementation_date <= '2023/10/01'
    GROUP by circumstances;
    """
cur.execute(sql)
result = cur.fetchall()
print(result)
circ_list = []
for circ in result:
    circ_list.append(circ[0])
if "B" in circ_list:
    j = True
else:
    j = False
print(j) 

[('A',), ('F',), ('T',)]
False


In [2]:
import sqlite3

DATABASE = r"database\database.db"

sql = f"""
    SELECT a.implementation_date,
    CASE
    WHEN a.circumstances = 'A' THEN '導入'
    WHEN a.circumstances = 'F' THEN '移動（元）'
    WHEN a.circumstances = 'T' THEN '移動（先）'
    WHEN a.circumstances = 'D' THEN '廃止'
    END as situation,
    a.department,
    b.name
    FROM T車両履歴 as a
    LEFT JOIN M部署 as b
    on a.department = b.code
    WHERE a.company_use_number = '216161'
    ORDER by id;
    """
conn = sqlite3.connect(DATABASE)
cur = conn.cursor()
cur.execute(sql)
result = cur.fetchall()
print(result)

[('2022/01/01', '導入', '03', '戸田'), ('2023/10/02', '移動（元）', '03', '戸田'), ('2023/10/02', '移動（先）', '07', '茨城')]


In [29]:
import sqlite3
import openpyxl

DATABASE = "database/database.db"

wb = openpyxl.load_workbook("月末指針_2022.xlsx")
sheetname_dict = {"茨城": "07", "足立": "09", "上三川": "22", "栃木": "02",
                  "名古屋": "13", "共配": "09", "千葉": "04", "神奈川": "08", "群馬": "05"}

rows = []
for k, v in sheetname_dict.items():
    ws = wb[k]
    i = 1
    c = 1
    for m in [4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3]:
        r = 3
        while True:
            if ws.cell(r, c).value is None:
                break
            else:
                section = (i, 2022, m,
                        ws.cell(r, c).value,
                        v,
                        0.0 if ws.cell(r, c+2).value is None else float(ws.cell(r, c+2).value),
                        0.0 if ws.cell(r, c+4).value is None else float(ws.cell(r, c+4).value))
                rows.append(section)
                i += 1
                r += 1
ws = wb["戸田"]
i = 1
c = 1
for m in [4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3]:
    r = 3
    while True:
        if ws.cell(r, c).value is None:
            break
        else:
            section = (i, 2022, m,
                    ws.cell(r, c).value,
                    "03",
                    0.0 if ws.cell(r, c+2).value is None else float(ws.cell(r, c+2).value),
                    0.0 if ws.cell(r, c+4).value is None else float(ws.cell(r, c+6).value))
            rows.append(section)
            i += 1
            r += 1

sql = """
    INSERT INTO T燃料使用量 VALUES(?, ?, ?, ?, ?, ?, ?)
    """
conn = sqlite3.connect(DATABASE)
cur = conn.cursor()
cur.executemany(sql, rows)
conn.commit()

In [6]:
import openpyxl

wb = openpyxl.load_workbook("月末指針_2022.xlsx")
ws = wb["茨城"]

print(type(ws.cell(29,1).value))

<class 'NoneType'>
